# Install dependencies

See [install_dependencies.ipynb](install_dependencies.ipynb).

# How to use this Notebook
1. Click *Kernel* -> *Restart Kernel and Run All Cells*.

# Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import io
import cv2
from matplotlib import pyplot as plt
import numpy as np
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, opencv_image_to_mpl, VideoDataGenerator, generate_image_bytes

In [ ]:
imp.reload(video);

# Define Pravega stream parameters

In [ ]:
gateway = 'hop-claudio-ubuntu18-desktop.solarch.lab.emc.com:54672'
scope = 'examples'
#stream = 'generated-video'
stream = 'object-detector-input-video'

# Initialize connection to Pravega GRPC Gateway

In [ ]:
pravega_channel = grpc.insecure_channel(gateway)
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

# Create data generator

In [ ]:
imp.reload(video);
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, IndexedStream, opencv_image_to_mpl, VideoDataGenerator

In [ ]:
height = int(450)
width = int(height * 16/9)
num_cameras = 1
frames_per_sec = 3.0
width, height

In [ ]:
video_data_generator = VideoDataGenerator(
    pravega_client, scope, stream, verbose=False,
    frames_per_sec=num_cameras*frames_per_sec, width=width, height=height, quality=92)

In [ ]:
#video_data_generator.truncate_stream()

# View sample generated image

In [ ]:
img_bytes = video_data_generator.generate_image_bytes(frame_number=12345)
serialized_event_len = 500 + int(len(img_bytes) * 4/3)
print('JPEG image size: %d bytes' % len(img_bytes))
print('Serialized event length: %d bytes' % serialized_event_len)
image_bytes_array = np.frombuffer(img_bytes, dtype=np.uint8)
image_array = cv2.imdecode(image_bytes_array, cv2.IMREAD_UNCHANGED)
plt.imshow(opencv_image_to_mpl(image_array));

In [ ]:
MB_per_sec = serialized_event_len * video_data_generator.frames_per_sec * 1e-6
serialized_event_len, MB_per_sec

# Generate video frames

In [ ]:
video_data_generator.write_generated_video(num_frames=10000)